In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch.utils.data import random_split


Interpolation methods

    Bilinear : takes the weighted average of four nearest pixels in the original image 
               to compute the value of a new pixel in the scaled image

    Nearest-neighbor: picks the pixel value of the nearest pixel in the input image and
                      uses it for the output pixel. It is the fastest but results in
                      blocky or pixelated output.

    Bicubic interpolation: This method is more computationally expensive than bilinear 
                           interpolation and is known to produce smoother output. 
                           It takes a weighted average of 16 surrounding pixels 
                           in a 4x4 pixel neighborhood of the input image.

    Lanczos interpolation: This method is a more advanced version of bicubic interpolation 
                           that is known to produce higher quality output
                           but is even more computationally expensive.
    
    interpolation
        0 -> Nearest-neighbor interpolation
        1 -> Bilinear interpolation
        2 -> Bicubic interpolation
        3 -> Lanczos interpolation


In [ ]:
moderate_glucoma_folder = '/home/dileepkumar/Z_ISL/AI-Workshop-Materials/Day 10/data/MODERATE-GLAUCOMA'
no_glucoma_folder = '/home/dileepkumar/Z_ISL/AI-Workshop-Materials/Day 10/data/NO-GLAUCOMA'
severe_glucoma_folder = '/home/dileepkumar/Z_ISL/AI-Workshop-Materials/Day 10/data/MODERATE-GLAUCOMA'
target_size = 224
interpolation_type = 2
normalization_mean = (0.5, 0.5, 0.5)
normalization_std = (0.5, 0.5, 0.5)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
resize_transform = transforms.Resize((target_size, target_size), interpolation=interpolation_type)
tensor_transform = transforms.ToTensor()
normalize_transform = transforms.Normalize(normalization_mean, normalization_std)
transform = transforms.Compose([
            resize_transform,
            tensor_transform,
            normalize_transform
            ]
        )

In [ ]:
image = Image.open('./data/MODERATE-GLAUCOMA/N116231_20170615_125121_Color_R_001.JPG')
image_tensor = transform(image)

In [ ]:
image_tensor.shape

In [ ]:
class DatasetFolderLoader(Dataset):
    def __init__(self, image_dir, label, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = os.listdir(image_dir)
        self.label = label
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, self.label

In [ ]:
class MyCustomDataset(object):
    def __init__(self, target_size_height, target_size_width):
        self.moderate_glucoma_folder = '/home/dileepkumar/Z_ISL/AI-Workshop-Materials/Day 10/data/MODERATE-GLAUCOMA/'
        self.no_glucoma_folder = '/home/dileepkumar/Z_ISL/AI-Workshop-Materials/Day 10/data/NO-GLAUCOMA/'
        self.severe_glucoma_folder = '/home/dileepkumar/Z_ISL/AI-Workshop-Materials/Day 10/data/SEVERE-GLAUCOMA/'
        self.target_size_height = target_size_height
        self.target_size_width = target_size_width
        self.interpolation_type = 2
        self.normalization_mean = (0.5, 0.5, 0.5)
        self.normalization_std = (0.5, 0.5, 0.5)
        self.transform = transforms.Compose([
            transforms.Resize((self.target_size_height, self.target_size_width), interpolation = self.interpolation_type),
            transforms.ToTensor(),
            transforms.Normalize(self.normalization_mean, self.normalization_std)
            ]
        )

    def load_data(self, label, folder):
        data = DatasetFolderLoader(folder, label, self.transform)
        train_size = int(0.8 * len(data))
        test_size = len(data) - train_size
        train_data, test_data = random_split(data, [train_size, test_size])
        return train_data, test_data
    
    def load_customdataset(self):
        """ 0: moderate_glucoma, 1: no_glucoma, 2: severe_glucoma"""
        moderate_glucoma_train, moderate_glucoma_test = self.load_data(0, self.moderate_glucoma_folder)
        no_glucoma_train, no_glucoma_test = self.load_data(1, self.no_glucoma_folder)
        severe_glucoma_train, severe_glucoma_test = self.load_data(2, self.severe_glucoma_folder)
        all_training_data = torch.utils.data.ConcatDataset([moderate_glucoma_train, no_glucoma_train, severe_glucoma_train])
        all_testing_data = torch.utils.data.ConcatDataset([moderate_glucoma_test, no_glucoma_test, severe_glucoma_test])
        return all_training_data, all_testing_data

In [ ]:
all_train, all_test = MyCustomDataset(224, 224).load_customdataset()

In [ ]:
len(all_train), len(all_test)

In [ ]:
class MyCustomCNN(nn.Module):
    def __init__(self):
        super(MyCustomCNN, self).__init__()
        self.input_depth = 3
        self.output_classes = 3
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 10, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(10, 6, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(6 * 112 * 112, 120),
            nn.ReLU(inplace=True),
            nn.Linear(120, 10),
            nn.ReLU(inplace=True),
            nn.Linear(10, 3),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        print(x.size())
        """
            After the first convolutional layer: 
                output size = (input size - kernel size + 2 * padding) / stride + 1 
                            = (224 - 3 + 2) / 1 + 1 = 224 x 224  x 10
            After the second convolutional layer: 
                output size = (input size - kernel size + 2 * padding) / stride + 1 
                            = (224 - 3 + 2) / 1 + 1 = 224 x 224 x 6
            After the maxpool layer: 
                output size = (input size - kernel size) / stride + 1 
                            = (224 - 2) / 2 + 1 = 112 x 112 x 6
        """
        x = x.view(-1, 6 * 112 * 112)
        x = self.fc_layers(x)
        return x



In [ ]:
model = MyCustomCNN().to(device)
print(model)

In [ ]:
def train(model, train_loader, criterion, optimizer, num_epochs):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, running_loss/len(train_loader)))

In [ ]:
all_train_data, all_test_data = MyCustomDataset(224, 224).load_customdataset()

train_loader = DataLoader(all_train_data, batch_size=30, shuffle=True)
# Create the custom CNN model
model = MyCustomCNN().to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10


In [ ]:
train(model, train_loader, criterion, optimizer, num_epochs)

# Define the path to save the model
model_path = 'my_model.pt'

# Save the trained model
torch.save(model.state_dict(), model_path)



In [ ]:
def test(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Create a DataLoader for the test dataset
test_loader = DataLoader(all_test_data, batch_size=30)

# Load the trained model
model = MyCustomCNN()
model.load_state_dict(torch.load('my_model.pth'))

# Test the model on the test dataset
test_acc = test(model, test_loader)

# Print the test accuracy
print('Test Accuracy:', test_acc)

In [ ]:
from sklearn.metrics import confusion_matrix

# Set the model to evaluation mode
model.eval()

# Initialize the true and predicted labels
true_labels = []
pred_labels = []

# Iterate through the test dataset
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)

        # Get the predicted labels
        _, predicted = torch.max(outputs.data, 1)

        # Add the true and predicted labels to the lists
        true_labels += labels.tolist()
        pred_labels += predicted.tolist()

# Calculate the confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

print(cm)